In [1]:
import requests
from datetime import datetime
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
MONTHS = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
    "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

In [3]:
def get_url(start, stop, step):
    return "https://ssd.jpl.nasa.gov/api/horizons.api?" + "&".join(f"{k}={v}" for k, v in {
        "EPHEM_TYPE": "'VECTORS'",
        "COMMAND": "'399'",
        "CENTER": "'500@10'",
        "START_TIME": f"'{start}'",
        "STOP_TIME": f"'{stop}'",
        "STEP_SIZE": f"'{step}'",
        "OBJ_DATA": "'NO'",
    }.items())

In [4]:
def get_ephem(start, stop, step):
    resp = requests.get(get_url(start, stop, step))
    return resp.json()["result"].split("\n")

In [5]:
def find_origin(start, stop, step):
    ephem = get_ephem(start, stop, step)
    state = 0
    prev_dt = None
    dt = None
    prev_y = None
    y = None
    for line in ephem:
        match state:
            case 0:
                if "$$SOE" in line:
                    state = 1
            case 1:
                if "$$EOE" in line:
                    break
                elif "A.D." in line:
                    prev_dt = dt
                    dt = " ".join(line.split()[3:5])
                elif "X =" in line:
                    y = line.split("=")[2]
                    y = float(y[:y.find("Z")].strip())
                    if prev_y is not None and prev_y < 0.0 and y > 0.0:
                        break
                    prev_y = y
    return prev_dt, dt, prev_y, y

In [6]:
def year_origin(year):
    prev_dt, dt, _, _ = find_origin(f"{year}-01-01", f"{year + 1}-01-01", "1 day")
    prev_dt, dt, prev_y, y = find_origin(prev_dt, dt, "1 minute")
    if abs(prev_y) < abs(y):
        return prev_dt
    else:
        return dt

In [7]:
def dt_to_iso(dt):
    for k, v in MONTHS.items():
        dt = dt.replace(k, v)
    dt = dt.replace(" ", "T")
    return dt[:dt.find(".")]

In [8]:
def origin_ts(year):
    origin = year_origin(year)
    dt_str = dt_to_iso(origin)
    dt = datetime.fromisoformat(dt_str)
    ts = dt.timestamp()
    return ts

In [9]:
data = {}
for year in range(1970, 2021, 5):
    ts = origin_ts(year)
    print(year, ts)
    data[year] = ts

1970 22895580.0
1975 180686040.0
1980 338476800.0
1985 496267320.0
1990 654057780.0
1995 811849260.0
2000 969640140.0
2005 1127430720.0
2010 1285220940.0
2015 1443011820.0
2020 1600802520.0


In [10]:
regr = LinearRegression()

In [11]:
X = np.array(sorted(data.keys())).reshape(-1, 1)

In [12]:
Y = np.array([data[x[0]] for x in X])

In [13]:
regr.fit(X, Y)

LinearRegression()

In [14]:
res = regr.predict(np.array([1970, 1971]).reshape(-1, 1))

In [15]:
sidereal = res[1] - res[0]
phase = res[0] / sidereal
2.0 * np.pi - phase * 2.0 * np.pi, sidereal

(1.7247443415579253, 31558144.36363983)

In [16]:
solstice = 1703215620.0
origin = 1600802520.0
axial_direction = (solstice - origin) % sidereal / sidereal * 2.0 * np.pi
axial_direction

1.54075846982669